In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Demo end-to-end de un agente

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

#! Crear el agente
memory = MemorySaver()
model = ChatOpenAI(model_name="gpt-4o-mini")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

#! Usar el agente
config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
): 
    print(chunk)
    print("----")

print("----"*3)

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="what is the weather in san francisco?")]}, config
): 
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hi Bob! It's great to meet you. How's life in San Francisco?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 88, 'total_tokens': 106, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-597063a4-55f2-4d84-a1e3-d4476e619dd8-0', usage_metadata={'input_tokens': 88, 'output_tokens': 18, 'total_tokens': 106, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
------------
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ovA7inUMwKySx8FfewRBjImL', 'function': {'arguments': '{"query":"San Francisc

#### B. Paso a paso

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("Cual es la temperatura en Chorrillos lima peru?")
print(search_results)

tools = [search]

[{'url': 'https://www.weatheravenue.com/es/america/pe/lima/chorrillos-tiempo.html', 'content': '☁ Previsiones meteorológicas de Chorrillos Lima Perú para 5 días. Hoy en Chorrillos Lima Perú: Ligeramente nublado con una temperatura de 19°C y un viento Suroeste con una velocidad de 12 Km/h. El grado de humedad será del 76% y habrá 0.0 mm de precipitaciones. ... Puesta del sol: 18:21 Salida de la luna : 04:53 - Puesta de la luna: 18'}, {'url': 'https://weather.com/es-US/tiempo/10dias/l/Chorrillos+Provincia+de+Lima+Perú?canonicalCityId=2fd625a064357d82aad8ee43b24d98072b1c1317be5b5b66de5020c6ddc7ca81', 'content': 'Consulta la temperatura máxima y mínima y la probabilidad de lluvia en The Weather Channel y Weather.com Pronóstico del tiempo para 10 días para Chorrillos, Provincia de Lima, Perú - The'}]


In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

#! Definir el modelo
model = ChatOpenAI(model_name="gpt-4o-mini")

#! Usar el modelo
response = model.invoke([HumanMessage(content="Hola!")])
print(response.content)

¡Hola! ¿Cómo puedo ayudarte hoy?


In [14]:
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hola!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: ¡Hola! ¿Cómo puedo ayudarte hoy?
ToolCalls: []


In [15]:
response = model_with_tools.invoke([HumanMessage(content="Cual es la temperatura en Chorrillos lima peru?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'temperatura Chorrillos Lima Peru'}, 'id': 'call_BDqZNXhXe2qGH9OWg0s3C7bN', 'type': 'tool_call'}]


In [17]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model_with_tools, tools)

response = agent_executor.invoke({"messages": [HumanMessage(content="Hola!")]})
response["messages"]

[HumanMessage(content='Hola!', additional_kwargs={}, response_metadata={}, id='2d90ca16-de9b-4fd1-ad8a-c54bda0762c7'),
 AIMessage(content='¡Hola! ¿Cómo puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-620900de-d4a6-44d0-9db0-70f7e0c6856d-0', usage_metadata={'input_tokens': 81, 'output_tokens': 11, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [19]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Cual es la temperatura en Lima Peru?")]})
response["messages"]

[HumanMessage(content='Cual es la temperatura en Lima Peru?', additional_kwargs={}, response_metadata={}, id='273c3a30-65be-49da-b4fa-5ed0a6534769'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bqvqAH1vRFx6LtyAtm2bimc9', 'function': {'arguments': '{"query":"temperatura en Lima Peru"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b11028c1-4b2a-4eb8-85f0-95b74a8f933b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'temperatura en Lima Peru'}, 'id': 'call_bqvqAH1vRFx6

In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Cual es la temperatura en Lima Peru?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WVcEYvsGGDsJ1jnU1QmudJz6', 'function': {'arguments': '{"query":"temperatura actual en Lima Peru"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0f280b02-6117-4d8b-aae4-621eb50a1bb1-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'temperatura actual en Lima Peru'}, 'id': 'call_WVcEYvsGGDsJ1jnU1QmudJz6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 11

#### C. Agente con memoria

In [22]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [27]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hola soy Nahum!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='¡Hola de nuevo, Nahum! ¿En qué puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 155, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-b27e0bb1-c64d-432b-96d1-6c164bbcee26-0', usage_metadata={'input_tokens': 155, 'output_tokens': 17, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [28]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Cual es mi nombre?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Tu nombre es Nahum. ¿Hay algo más en lo que te pueda ayudar?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 184, 'total_tokens': 203, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d441edd-e08c-41a6-97c8-766386b61692-0', usage_metadata={'input_tokens': 184, 'output_tokens': 19, 'total_tokens': 203, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [29]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Cual es mi nombre?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='No tengo acceso a tu información personal, así que no sé tu nombre. Si deseas, puedes decírmelo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 123, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-fbfebf6a-50e9-41ac-bb8e-7110fa1203d6-0', usage_metadata={'input_tokens': 123, 'output_tokens': 26, 'total_tokens': 149, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
